In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
from torch.autograd import Variable
from torch.nn.utils import weight_norm

In [3]:
%run utils.ipynb
#from utils import *
%run config.ipynb
#from config import *

In [4]:
class JointNet(nn.Module):
    def __init__(self):
        super(JointNet, self).__init__()
        self.num_class = 345
        self.tokens_range = 256
        self.embedding_size = 512
        self.feature_size = 512
        self.max_length = 200
        self.img_size = 28
        self.dropout = 0.3
        # RNN Model
        self.embed_net = EmbeddingNet(self.tokens_range, self.embedding_size, self.feature_size, self.dropout)
        # CNN Model
        self.cnn_model = FCModel(self.img_size*self.img_size, self.feature_size, self.dropout)
        # Classification Model
        self.classifier = FCModel(self.feature_size * 2, self.num_class, self.dropout)
    
    def forward(self, image_input, sequence_input):
        # input-> image: [batch_size, 28, 28]
        # input-> sequence: [batch_size, MAXI_LENGTH, 2]
        
        # output size [batch_size, 345]
        rnn_feat = self.embed_net(sequence_input)
        cnn_feat = self.cnn_model(image_input)
        
        output = self.classifier(torch.cat((rnn_feat, cnn_feat), 1))
        
        return output

In [5]:
class EmbeddingNet(nn.Module):
    def __init__(self, tokens_range, embedding_size, feature_size, drop):
        super(EmbeddingNet, self).__init__()
        self.embedding_x = nn.Embedding(tokens_range, embedding_size, padding_idx=0)
        self.embedding_y = nn.Embedding(tokens_range, embedding_size, padding_idx=0)
        self.drop = nn.Dropout(drop)
        self.nonliner = nn.Sigmoid()
        # generate lstm
        self.lstm = nn.GRU(input_size=embedding_size,
                           hidden_size=int(feature_size/2),
                           num_layers=2,
                           dropout = drop,
                           batch_first=True)
        self._init_lstm(self.lstm.weight_ih_l0)
        self._init_lstm(self.lstm.weight_hh_l0)
        self._init_lstm(self.lstm.weight_ih_l1)
        self._init_lstm(self.lstm.weight_hh_l1)
        self.lstm.bias_ih_l0.data.zero_()
        self.lstm.bias_hh_l0.data.zero_()
        self.lstm.bias_ih_l1.data.zero_()
        self.lstm.bias_hh_l1.data.zero_()
    def _init_lstm(self, weight):
        for w in weight.chunk(3, 0):
            init.xavier_uniform_(w)
    def forward(self, input_feat):
        # input size [batch_size, MAXI_LENGTH, 2]
        # input value: 1-254: position, 0: default, 1: end of stroke
        batch_size, input_length, xy = input_feat.shape

        input_x = input_feat[:,:,0].contiguous().view(batch_size, input_length)
        input_y = input_feat[:,:,1].contiguous().view(batch_size, input_length)
        
        embedded_x = self.embedding_x(input_x)
        embedded_y = self.embedding_y(input_y)
        
        joint_embedded = self.nonliner(embedded_x) * self.nonliner(embedded_y)
        _, output = self.lstm(joint_embedded)
        
        return torch.transpose(output, 0, 1).contiguous().view(batch_size,-1)

In [6]:
class FCModel(nn.Module):
    def __init__(self, input_size, output_size, drop):
        super(FCModel, self).__init__()
        self.input_size = input_size
        self.fc1 = nn.Linear(input_size, 1024)
        self.bn1 = nn.BatchNorm1d(1024)
        self.fc2 = nn.Linear(1024, 1024)
        self.bn2 = nn.BatchNorm1d(1024)
        self.fc3 = nn.Linear(1024, output_size)
        self.relu = nn.ReLU()
        self.drop = nn.Dropout(drop)
    def forward(self, input_feat):
        input_feat = input_feat.view(-1, self.input_size)
        x = self.fc1(input_feat)
        x = self.relu(x)
        x = self.bn1(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.bn2(x)
        x = self.fc3(x)
        return x